In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
data = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")

In [ ]:
data.head()

In [ ]:
base_image_dir_path = "../input/plant-pathology-2021-fgvc8/train_images/"

In [ ]:
data["image_path"] = [f"{base_image_dir_path}{image_id}" for image_id in data["image"].values]

In [ ]:
data.head()

In [ ]:
# Counting the unique values for label and double crossing
data["image"].loc[0], data["image_path"].loc[0]

In [ ]:
target_dir = {value : key for key,value in enumerate(list(data["labels"].unique()))}

In [ ]:
target_dir

In [ ]:
data["labels"] = data["labels"].map(target_dir)

In [ ]:
data.head()

In [ ]:
(*IMAGE_SIZE,3)

In [ ]:
# image resolution for eff net model
"""
Base model	resolution
EfficientNetB0	224
EfficientNetB1	240
EfficientNetB2	260
EfficientNetB3	300
EfficientNetB4	380
EfficientNetB5	456
EfficientNetB6	528
EfficientNetB7	600

"""

AUTO = tf.data.experimental.AUTOTUNE

# Configuration
IMAGE_SIZE = [260, 260]
EPOCHS = 40
SEED = 24
BATCH_SIZE = 32 


def prepare_images(image, label):
    img = tf.io.read_file(image)
    # Decode Jpeg files
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, size = IMAGE_SIZE, method = "bicubic")
    
    return img,label

def augment_image(image,label):
    img = tf.image.random_flip_up_down(image, seed = SEED)
    img = tf.image.rot90(img)
    img = tf.image.flip_left_right(img)

    
    return img, label


def decode_tensor_to_image(tensor):
    if len(tensor.shape) >= 4:
        if tensor.shape[0] == 1:
            tensor = tf.squeeze(tensor, axis = 0)
        else:
            tensor = tensor[0]
    img = tf.keras.preprocessing.image.array_to_img(tensor)
    
    return img

def process_function(image,label):
    img = tf.keras.applications.efficientnet.preprocess_input(image)
    return img, label

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_files , valid_files = train_test_split(data, test_size = 0.1)

In [ ]:
def get_dataset(files, batch_size):
    image_dataset = tf.data.Dataset.from_tensor_slices(files["image_path"].values)
    labels_dataset = tf.data.Dataset.from_tensor_slices(files["labels"].values)
    final_dataset = tf.data.Dataset.zip((image_dataset, labels_dataset))
    final_dataset = final_dataset.shuffle(1000)
    final_dataset = final_dataset.map(prepare_images, num_parallel_calls = AUTO)
    final_dataset = final_dataset.map(augment_image, num_parallel_calls = AUTO)
    final_dataset = final_dataset.map(process_function, num_parallel_calls = AUTO)
    final_dataset = final_dataset.batch(batch_size, drop_remainder = False)
    final_dataset = final_dataset.prefetch(4)
    
    return final_dataset

In [ ]:
train_dataset = get_dataset(train_files, batch_size = 32)
valid_dataset = get_dataset(valid_files, batch_size = 24)

In [ ]:
img, label = next(iter(valid_dataset))
image = decode_tensor_to_image(img)
image

# Model Building

In [ ]:
model = keras.applications.EfficientNetB2(include_top = True,weights = None, input_shape = (*IMAGE_SIZE,3) , classes = 12, classifier_activation = "softmax")
model.summary()

In [ ]:
model.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy(), metrics = ["accuracy"])

In [ ]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = "./model",monitor = "val_accuracy", mode = "max",save_best_only = True)

In [ ]:
model.fit(train_dataset, epochs = EPOCHS, callbacks = model_checkpoint, validation_data = valid_dataset)